In [1]:
!pip install scikit-multilearn
!pip install contractions
# !pip install mlxtend

     |████████████████████████████████| 89 kB 7.8 MB/s 
     |████████████████████████████████| 106 kB 35.4 MB/s 
     |████████████████████████████████| 287 kB 54.4 MB/s 


In [2]:
%pip install mlxtend --upgrade

     |████████████████████████████████| 1.3 MB 24.0 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [3]:
import pandas as pd
import numpy as np
import re
import nltk.corpus
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords, words
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, MinMaxScaler
from skmultilearn.problem_transform import BinaryRelevance, LabelPowerset, ClassifierChain
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from mlxtend.evaluate import bias_variance_decomp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Reading dataset

In [4]:
test_label = pd.read_csv("test_labels.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [5]:
# Merging test and train to form one huge dataset
test_data = pd.merge(test, test_label)
dataset = pd.concat([test_data, train])
dataset.drop(columns=['id'], inplace=True)
dataset.drop_duplicates(inplace=True, ignore_index=True)
dataset.drop(dataset.index[dataset['toxic'] == -1], inplace = True)
dataset.reset_index(inplace = True)

## Text cleaning

In [6]:
# Text cleaning
#converting to lower case
dataset['comment_text_cleaned'] = dataset['comment_text'].str.lower()
#removing special characters
dataset['comment_text_cleaned'] = dataset['comment_text_cleaned'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", str(elem)))
#removing numbers
dataset['comment_text_cleaned'] = dataset['comment_text_cleaned'].apply(lambda elem: re.sub(r"\d+", "", str(elem)))
# Removing stop words
stop = stopwords.words('english')
dataset['comment_text_cleaned'] = dataset['comment_text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# Replacing contractions with their full forms
dataset['comment_text_cleaned'] = dataset['comment_text_cleaned'].apply(lambda x: contractions.fix(x))
#Tokenizing
dataset['comment_text_cleaned'] = dataset['comment_text_cleaned'].apply(lambda x: word_tokenize(x))
#Lemmitization
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text
dataset['comment_text_cleaned'] = dataset['comment_text_cleaned'].apply(lambda x: word_lemmatizer(x))

# Splitting dataset

In [7]:
# Splitting into train test sets
X = dataset.drop(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
y = dataset[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].copy()

X_train, X_test_and_val, y_train, y_test_and_val = train_test_split(X,y, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_test_and_val,y_test_and_val, train_size=0.5)

# Word to Vec

In [8]:
train_tokens = pd.Series(X_train['comment_text_cleaned']).values
w2v_model = Word2Vec(train_tokens, size= 200)

def buildWordVector(tokens, size):
  vec = np.zeros(size).reshape((1, size))
  count = 0.
  for word in tokens:
    try:
      vec += w2v_model[word].reshape((1, size))
      count += 1.
    except KeyError:
      continue
  if count != 0:
    vec /= count
  return vec

In [9]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in train_tokens])
# train_vecs_w2v = scaler.fit_transform(train_vecs_w2v)
train_vecs_w2v = scale(train_vecs_w2v)

val_tokens = pd.Series(X_val['comment_text_cleaned']).values
val_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in val_tokens])
# val_vecs_w2v = scaler.transform(val_vecs_w2v)
val_vecs_w2v = scale(val_vecs_w2v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [10]:
def evaluation_metric(model_name, feature_extraction, y_true, y_pred):
  print('Model:', model_name)
  print('Feature extraction method:', feature_extraction)
  recall= recall_score(y_true, y_pred, average='micro')
  prec = precision_score(y_true, y_pred, average='micro')
  final_score = recall*0.6 + prec*0.4
  print('Precision: ', prec)
  print('Recall: ', recall)
  print('Final score of the model: ', final_score)
  return final_score

# Logistic Regression + Classifier Chain

In [ ]:
classifier = ClassifierChain(LogisticRegression(max_iter = 100))
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Logistic Regression with Classifier Chain', 'Word2Vec', y_val, predictions)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


NameError: ignored

In [ ]:
predictions = classifier.predict(train_vecs_w2v)
evaluation_metric('Logistic Regression with Classifier Chain', 'Word2Vec', y_train, predictions)

NameError: ignored

In [ ]:
mse, bias, var = bias_variance_decomp(classifier, np.array(train_vecs_w2v), y_train.values, np.array(val_vecs_w2v), y_val.values, loss='mse', num_rounds=200, random_seed=123)

In [ ]:
print('MSE from bias_variance lib [avg expected loss]: %.3f' % mse)
print('Avg Bias: %.3f' % bias)
print('Avg Variance: %.3f' % var)

NameError: ignored

# Naive Bayes + Bianary Relevance

In [ ]:
classifier = BinaryRelevance(GaussianNB())
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)

In [ ]:
df = pd.DataFrame(predictions.toarray(), columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
df[df.eq(1).any(1)]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,0,0,1,0,0
1,1,0,1,0,1,1
5,1,1,1,1,1,1
6,1,0,1,0,1,0
14,1,0,1,0,1,0
...,...,...,...,...,...,...
22310,1,0,1,1,1,1
22311,1,1,1,1,1,1
22316,1,1,1,1,1,1
22321,1,1,1,0,1,1


In [ ]:
print("Score = ",evaluation_metric(y_val, predictions))

TypeError: ignored

# Logistic Regression + Binary Relevance

In [ ]:
classifier = BinaryRelevance(LogisticRegression(max_iter=100))
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)

In [ ]:
print("Score = ",evaluation_metric(y_val, predictions.todense()))

TypeError: ignored

In [ ]:
classifier = LabelPowerset(GaussianNB())
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)

In [ ]:
print("Score = ",evaluation_metric(y_val, predictions))

0.14712725065749546
0.6373794916739702
Score =  None


# Nueral Network

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim = train_vecs_w2v.shape[1], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',)
model.fit(train_vecs_w2v, y_train, epochs=100, verbose=0)

In [ ]:
predictions = model.predict(val_vecs_w2v)
predictions = predictions.round()
predictions

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
evaluation_metric('Neural Network', 'Word2Vec', y_val, predictions)

Model: Neural Network
Feature extraction method: Word2Vec
Recall: 0.8687648456057007
Precision 0.07589486425730589	(Too Low)


In [ ]:
sizes, training_scores, testing_scores = learning_curve(LabelPowerset(GaussianNB()), train_vecs_w2v, y_train, cv=10, scoring='recall_micro', train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
# Mean and Standard Deviation of training scores
mean_training = np.mean(training_scores, axis=1)
Standard_Deviation_training = np.std(training_scores, axis=1)
  
# Mean and Standard Deviation of testing scores
mean_testing = np.mean(testing_scores, axis=1)
Standard_Deviation_testing = np.std(testing_scores, axis=1)
  
# dotted blue line is for training scores and green line is for cross-validation score
plt.plot(sizes, mean_training, '--', color="b",  label="Training score")
plt.plot(sizes, mean_testing, color="g", label="Cross-validation score")
  
# Drawing plot
plt.title("LEARNING CURVE FOR KNN Classifier")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

NameError: ignored

# Comparing Feature extraction Techniques by using Naive Bayes with Classifier Chains

##Bag of words

In [ ]:
def evaluation_metric(model_name, feature_extraction, y_true, y_pred):
  print('Model:', model_name)
  print('Feature extraction method:', feature_extraction)
  print('Recall:', recall_score(y_true, y_pred, average='micro'))
  prec = precision_score(y_true, y_pred, average='micro')
  print('Precision', prec, end = '')
  if(prec < 0.2):
    print('\t(Too Low)')

In [ ]:
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False, max_features = 1000)
X_tr_bow = bow_transform.fit_transform(X_train['comment_text_cleaned'])
X_te_bow = bow_transform.transform(X_val['comment_text_cleaned'])

In [ ]:
classifier = ClassifierChain(GaussianNB())
classifier.fit(X_tr_bow, y_train)
predictions = classifier.predict(X_te_bow)
evaluation_metric('Naive Bayes with Classifier Chain', 'Bag of Words', y_val, predictions)

Model: Naive Bayes with Classifier Chain
Feature extraction method: Bag of Words
Recall: 0.8687648456057007
Precision 0.07589486425730589	(Too Low)


## Tf-idf

In [ ]:
tfidf_transform = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_transform.fit_transform(X_tr_bow)
X_te_tfidf = tfidf_transform.transform(X_te_bow)

In [ ]:
classifier = ClassifierChain(GaussianNB())
classifier.fit(X_tr_tfidf, y_train)
predictions = classifier.predict(X_te_tfidf)
evaluation_metric('Naive Bayes with Classifier Chain', 'TF-IDF', y_val, predictions)

Model: Naive Bayes with Classifier Chain
Feature extraction method: TF-IDF
Recall: 0.8580542264752791
Precision 0.07754396079561833	(Too Low)


## word2vec

In [ ]:
classifier = ClassifierChain(GaussianNB())
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Naive Bayes with Classifier Chain', 'Word2Vec', y_val, predictions)

Model: Naive Bayes with Classifier Chain
Feature extraction method: Word2Vec
Recall: 0.7741228070175439
Precision 0.22799600728084082

# Comparing Multilabel Classification techniques with Logistic Regression + Word2Vec (Ran for  Naive Bayes Too) 

##Binary relevance

In [20]:
classifier = BinaryRelevance(LogisticRegression())
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Logistic Regression with Binary Relevance', 'Word2Vec', y_val, predictions)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Model: Logistic Regression with Binary Relevance
Feature extraction method: Word2Vec
Precision:  0.7482344632768362
Recall:  0.4468578658793758
Final score of the model:  0.5674085048383599


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.5674085048383599

## Label Powerset

In [21]:
classifier = LabelPowerset(LogisticRegression())
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Logistic Regression with Label Powerset', 'Word2Vec', y_val, predictions)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Model: Logistic Regression with Label Powerset
Feature extraction method: Word2Vec
Precision:  0.755637707948244
Recall:  0.43104175453395194
Final score of the model:  0.5608801358996688


0.5608801358996688

##Classifier Chains

In [22]:
classifier = ClassifierChain(LogisticRegression())
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Logistic Regression with Classifier Chain', 'Word2Vec', y_val, predictions)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Model: Logistic Regression with Classifier Chain
Feature extraction method: Word2Vec
Precision:  0.7055053888727061
Recall:  0.5107549557148883
Final score of the model:  0.5886551289780154


0.5886551289780154

## Saving Tuned Naive Bayes (Best parameters chosen after generating score v/s value graphs for hyperparamters) 

In [14]:
classifier = ClassifierChain(GaussianNB(var_smoothing = 0.0))
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Naive Bayes with Classifier Chain', 'Word2Vec', y_val, predictions)
import pickle
filename = 'tuned_naive_bayes_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:489: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:490: RuntimeWarning: divide by zero encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:490: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:490: RuntimeWarning: invalid value encountered in subtract
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


Model: Naive Bayes with Classifier Chain
Feature extraction method: Word2Vec
Precision:  0.04405732302728011
Recall:  0.8810628426824125
Final score of the model:  0.5462606348203595


## Saving Tuned Logistic Regression (Best parameters chosen after generating score v/s value graphs for hyperparamters) 

In [16]:
classifier = ClassifierChain(LogisticRegression(solver = 'newton-cg', C = 2000, max_iter = 200))
classifier.fit(train_vecs_w2v, y_train)
predictions = classifier.predict(val_vecs_w2v)
evaluation_metric('Logistic with Classifier Chain', 'Word2Vec', y_val, predictions)
import pickle
filename = 'tuned_logistic_regression_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

Model: Naive Bayes with Classifier Chain
Feature extraction method: Word2Vec
Precision:  0.7056768558951965
Recall:  0.5111767186840995
Final score of the model:  0.5889767735685383
